## **Setting up the Environment**

In [1]:
% cd /content/drive/MyDrive/Colab Notebooks/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/HASOC Tamil/Task 1


## **Importing Libraries**

In [53]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

## **Loading Data**

In [3]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [4]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [5]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam4981,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [6]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [7]:
dataframe.shape

(5880, 3)

In [8]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [9]:
dataframe.shape

(5877, 3)

In [10]:
text = dataframe['text']

In [11]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [12]:
label = dataframe['category']

In [13]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [14]:
label = LabelEncoder().fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [15]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

## **Feature Extraction**

In [32]:
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(text)
X = X.todense()

In [36]:
X.shape

(5877, 782)

In [37]:
y = label
y.shape

(5877,)

## **Test Train Split**

In [39]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
X_train.shape

(4701, 782)

In [41]:
X_val.shape

(1176, 782)

## **Logistic Regression**

In [43]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

              precision    recall  f1-score   support

           0       0.83      0.97      0.90       950
           1       0.60      0.18      0.28       226

    accuracy                           0.82      1176
   macro avg       0.72      0.58      0.59      1176
weighted avg       0.79      0.82      0.78      1176



## **Support Vector Machine**

In [45]:
svm = SVC()
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.82      0.99      0.90       950
           1       0.67      0.10      0.17       226

    accuracy                           0.82      1176
   macro avg       0.74      0.54      0.53      1176
weighted avg       0.79      0.82      0.76      1176



## **Naive Bayes**

In [46]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.88      0.24      0.37       950
           1       0.21      0.86      0.34       226

    accuracy                           0.36      1176
   macro avg       0.54      0.55      0.36      1176
weighted avg       0.75      0.36      0.37      1176



## **Stochastic Gradient Descent**

In [47]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.83      0.97      0.90       950
           1       0.57      0.17      0.27       226

    accuracy                           0.82      1176
   macro avg       0.70      0.57      0.58      1176
weighted avg       0.78      0.82      0.77      1176



## **K Nearest Neighbours**

In [48]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89       950
           1       0.36      0.04      0.06       226

    accuracy                           0.80      1176
   macro avg       0.59      0.51      0.48      1176
weighted avg       0.73      0.80      0.73      1176



## **Decision Tree**

In [49]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85       950
           1       0.36      0.33      0.34       226

    accuracy                           0.76      1176
   macro avg       0.60      0.60      0.60      1176
weighted avg       0.75      0.76      0.76      1176



## **Random Forest**

In [50]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.82      0.98      0.90       950
           1       0.62      0.11      0.18       226

    accuracy                           0.82      1176
   macro avg       0.72      0.55      0.54      1176
weighted avg       0.78      0.82      0.76      1176



## **Majority Voting**

In [57]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Removing KNN and NB from predictions due to poor performance
    predictions = [lr_pred[i], svm_pred[i], sgd_pred[i], dt_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))

              precision    recall  f1-score   support

           0       0.83      0.98      0.90       950
           1       0.65      0.12      0.21       226

    accuracy                           0.82      1176
   macro avg       0.74      0.55      0.55      1176
weighted avg       0.79      0.82      0.77      1176



## **Loading Testing Data**

In [65]:
dataframe_test = pd.read_csv("Dataset/tam_offesive_withoutlabels_test.tsv", sep='\t')
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [66]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_642,யாரு எல்லாம் விஸ்வாசம் டிரெய்லர் பாத்துட்டு வந...
freq,1,1


In [68]:
text_test = dataframe_test['text']

0      தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1      நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2                         தில்லானா முயற்சி  தஞ்சை கோனார்
3      திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4      நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...
                             ...                        
649    நான் தலைவரோட ரசிகன்னு சொல்லிடு தலைவரை ஒடசல் போ...
650    வாழ்த்துக்கள்,உண்மை கதை உலகத்துக்கு காட்டிய மோ...
651    குருமா and பா..குஞ்சித் பயலுக கதர்ரது இன்னும் ...
652    வெள்ளார்கள் சார்பாக படம் வெற்றி பெற வாழ்த்துக்...
653    சமீபகாலமாக கால்பந்தாட்டத்தை மையப்படுத்தி தமிழ்...
Name: text, Length: 654, dtype: object

## **Test Data Preprocessing**

In [69]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

## **Test Feature Extraction**

In [70]:
vectorizer = TfidfVectorizer(min_df = 5)
X_test = vectorizer.fit_transform(text)
X_test = X_test.todense()

In [71]:
X_test.shape

(5877, 782)